In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
days=50

In [ ]:
timePriod = f"{days}d"

In [ ]:

indexDict = {
    "Nifty Data": "^NSEI",
    "Sensex Data": "^BSESN",
    "Bank Nifty Data": "^NSEBANK",
    "IT Sector Data": "^CNXIT",
    "Auto Sector Data": "^CNXAUTO",
    "Metal Sector Data": "^CNXMETAL",
    "Realty Sector Data": "^CNXREALTY",
    "FMCG Sector Data": "^CNXFMCG",
    "Pharma Sector Data": "^CNXPHARMA",
    "Energy Sector Data": "^CNXENERGY",
    "Gold Data": "GC=F",
    "Crude Oil Data": "CL=F",
    "USD-INR Data": "INR=X",
    "India VIX Data": "^INDIAVIX"
}
dataDict = {}

In [ ]:
# the index and data finding
for name , symbol in indexDict.items():
    dataDict[name] = yf.Ticker(ticker=symbol).history(period=timePriod)

In [ ]:
# the data shapes
for names , data in dataDict.items():
    print(f'{names} data shape : ', data.shape)


In [ ]:
dataDict['IT Sector Data'].tail()

In [ ]:
# finding the null values
for names ,data in dataDict.items():
    display(f'{names} the data : ', data.isnull().sum())

In [ ]:
# finding the null values
for names ,data in dataDict.items():
    data.dropna()
dataDict.__len__()

In [ ]:
import numpy as np
import pandas as pd

riskDict = {}

for name, df in dataDict.items():
    risk_df = pd.DataFrame()
    
    # 1. Setup Dates
    if isinstance(df.index, pd.DatetimeIndex):
        risk_df['Date'] = df.index
    else:
        date_col = [c for c in df.columns if 'date' in c.lower()][0]
        risk_df['Date'] = pd.to_datetime(df[date_col])
    
    risk_df = risk_df.sort_values('Date').reset_index(drop=True)
    price_col = 'Adj Close' if 'Adj Close' in df.columns else 'Close'
    
    # 2. Calculate Returns
    risk_df['log_return'] = np.log(df[price_col] / df[price_col].shift(1)).values

    # 3. Rolling Volatility with faster warmup (min_periods=10)
    for w in [25, 90, 252,500, 600,900,1000,1200]:
        if len(risk_df) < 10: continue
        
        vol_col = f'vol_{w}d'
        risk_df[vol_col] = risk_df['log_return'].rolling(window=w, min_periods=10).std() * np.sqrt(252)

        # 4. Robust Z-Score (MAD)
        # We use a shorter z_window if 252 days aren't available yet
        z_win = min(len(risk_df), 252)
        median = risk_df[vol_col].rolling(z_win, min_periods=10).median()
        mad = (risk_df[vol_col] - median).abs().rolling(z_win, min_periods=10).median()
        
        risk_df[f'{vol_col}_z'] = (risk_df[vol_col] - median) / (1.4826 * mad + 1e-9)

    # 5. Composite Score - Only calculate if columns exist
    z_cols = [c for c in risk_df.columns if c.endswith('_z')]
    if z_cols:
        # This ensures we don't get 0.00 if data is missing; we get NaN instead
        risk_df['crash_risk_score'] = risk_df[z_cols].mean(axis=1)
    
    riskDict[name] = risk_df

# --- Aggregation ---
all_scores = [r_df.set_index('Date')['crash_risk_score'] for r_df in riskDict.values()]
market_df = pd.concat(all_scores, axis=1).mean(axis=1).to_frame(name='market_score')

# --- THE ACCURACY CHECK ---
current_risk = market_df['market_score'].iloc[-1]

print(f"--- FINAL CRASH RISK REPORT ---")
print(f"Current Market Risk Score: {current_risk:.2f}")

if np.isnan(current_risk):
    print("STATUS: INSUFFICIENT DATA (Need at least 10-20 days of price history)")
elif current_risk >= 3.0:
    print("STATUS: EXTREME CRASH RISK - Volatility is in the 99th percentile.")
elif current_risk >= 2.0:
    print("STATUS: HIGH RISK - Potential correction or crash incoming.")
elif current_risk >= 1.5:
    print("STATUS: ELEVATED - Market is becoming unstable.")
else:
    print("STATUS: NORMAL - No immediate crash detected based on volatility clusters.")

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(25, 20))

for ax, (key, values) in zip(axes.flatten(), dataDict.items()):
    ax.plot(values['High'], '-o')
    ax.plot(values['Low'],'-x')
    ax.set_title(key)
plt.show()



In [ ]:
dataDict['IT Sector Data'].to_csv(
    '../../DATA/ITDATA.CSV',
    index=False,
    mode='w'   # force overwrite
)
